In [ ]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt

## 1. 데이터 로드 및 기본 전처리

In [ ]:
# 분류 결과가 저장된 pickle 파일 로드
df_gpt_result = pd.read_pickle('./classify_result/df_result_final.pickle')

# 'classify_result' 컬럼이 리스트 형태가 아닌 경우 필터링
df_gpt_result = df_gpt_result[df_gpt_result['classify_result'].apply(lambda x: isinstance(x, list))].copy()
df_gpt_result = df_gpt_result[df_gpt_result['classify_result'].apply(len) > 0].reset_index(drop=True)

## 2. 분류 결과 데이터셋 생성

In [ ]:
# 'classify_result' 리스트를 개별 행으로 분리 (explode)
df_exploded = df_gpt_result.explode('classify_result').dropna().reset_index(drop=True)

# 리스트 원소의 길이가 3이 아닌 잘못된 형식의 데이터 처리
def extract_data(row):
    if isinstance(row, list) and len(row) == 3:
        return pd.Series(row, index=['review_type', 'review_comment', 'review_satisfy'])
    return pd.Series([np.nan, np.nan, np.nan], index=['review_type', 'review_comment', 'review_satisfy'])

# 분리된 'classify_result'에서 속성, 내용, 만족도 추출
extracted_cols = df_exploded['classify_result'].apply(extract_data)

# 원본 데이터프레임과 추출된 컬럼 병합
df_processed = pd.concat([df_exploded.drop(columns=['classify_result']), extracted_cols], axis=1)
df_processed.dropna(subset=['review_comment', 'review_type'], inplace=True)
df_processed = df_processed[df_processed['review_comment'].str.len() > 5].reset_index(drop=True)

## 3. 데이터 정제

In [ ]:
# 리뷰 속성(review_type) 값 통일
type_replace_map = {
    '가성비': '가격', '할인': '가격',
    '디스플레이': '화면', '화면 크기': '화면', '화질': '화면',
    '무게감': '무게',
    '배터리 수명': '배터리',
    '사용': '사용성', '사용성': '기능',
    '사은품': '구성', '케이스': '구성', '충전기': '구성',
    '사이즈': '크기',
    '사진': '카메라', '카메라 성능': '카메라',
    '속도': '성능',
    '자급제폰': '자급제',
    '컬러': '색상'
}
df_processed['review_type'] = df_processed['review_type'].replace(type_replace_map)

# 10회 미만으로 언급된 속성 제거
type_counts = df_processed['review_type'].value_counts()
valid_types = type_counts[type_counts > 10].index.tolist()
df_processed = df_processed[df_processed['review_type'].isin(valid_types)].reset_index(drop=True)

In [ ]:
# 만족/불만족(review_satisfy) 값 통일
satisfy_map = {
    1.0: '만족', 1: '만족', '1': '만족', '+1': '만족', '+': '만족', 
    'positive': '만족', 'Positive': '만족', '좋습니다.': '만족',
    -1.0: '불만족', -1: '불만족', '-1': '불만족',
    'negative': '불만족', 'Negative': '불만족', 'dissatisfaction': '불만족',
    '0': '보통', 0: '보통', 'neutral': '보통'
}
df_processed['review_satisfy'] = df_processed['review_satisfy'].replace(satisfy_map)
df_processed['review_satisfy'].fillna('만족', inplace=True)

# 정의되지 않은 값 확인 (있을 경우 추가 처리 필요)
# print(df_processed[~df_processed['review_satisfy'].isin(['만족', '불만족', '보통'])]['review_satisfy'].unique())

## 4. 최종 데이터셋 저장 (리뷰 속성 분류) 

In [ ]:
# 최종 데이터프레임 저장
df_processed.reset_index(inplace=True)
df_processed.rename(columns={'index': 'classify_review_id'}, inplace=True)
df_processed.to_pickle('./df_gpt_result_preprocessed.pickle')

# 대시보드 정렬용 데이터프레임 저장
df_review_type_count_for_sort = df_processed['review_type'].value_counts().reset_index()
df_review_type_count_for_sort.to_pickle('./df_review_type_count_for_sort.pickle')

## 5. 키워드 추출 데이터셋 생성

In [ ]:
# Okt 형태소 분석기 초기화
okt = Okt()

# 리뷰 내용에서 명사 추출
df_processed['review_words'] = df_processed['review_comment'].apply(lambda x: okt.nouns(str(x)) if pd.notna(x) else [])

In [ ]:
# 추출된 명사를 개별 행으로 분리
df_wordset = df_processed[['classify_review_id', 'review_words']].explode('review_words')
df_wordset.rename(columns={'review_words': 'word'}, inplace=True)

# 한 글자 단어(‘폰’ 제외) 및 불필요한 단어 제거
df_wordset = df_wordset[(df_wordset['word'].str.len() > 1) | (df_wordset['word'] == '폰')]
df_wordset['word'] = df_wordset['word'].replace('자급', '자급제')

In [ ]:
# 단어 동의어 처리
replace_list = [
    ['감상', '감사'], ['감샴당', '감사'], ['갤럭', '갤럭시'], ['갤릭시', '갤럭시'],
    ['검은색', '검정색'], ['검정', '검정색'], ['금제', '요금제'], ['데이타', '데이터'],
    ['듭니', '듭니다'], ['때매', '때문'], ['때메', '때문'], ['만족감', '만족'],
    ['만족도', '만족'], ['미개', '미개봉'], ['번가', '11번가'], ['별루', '별로'],
    ['부모', '부모님'], ['불량품', '불량'], ['뽁뽁', '뽁뽁이'], ['사은', '사은품'],
    ['살작', '살짝'], ['어머님', '어머니'], ['어머님께', '어머니'], ['어플리케이션', '어플'],
    ['역쉬', '역시'], ['울트랄', '울트라'], ['유투브', '유튜브'], ['유툽', '유튜브'],
    ['유튜', '유튜브'], ['유튜버들', '유튜버'], ['진짜루', '진짜'], ['진짜진짜', '진짜'],
    ['최신기', '최신'], ['최신기종', '최신'], ['파란색', '파란'], ['파랑', '파란'],
    ['파랑색', '파란'], ['핑쿠', '핑크'], ['핑쿠색', '핑크'], ['핑크색', '핑크'],
    ['하늘', '하늘색'], ['하늘빛', '하늘색'], ['휴대전화', '휴대폰']
]

In [ ]:
for old, new in replace_list:
    df_wordset['word'] = df_wordset['word'].replace(old, new)

In [ ]:
# 최종 키워드 데이터셋 저장
df_wordset.to_pickle('./df_classify_wordset.pickle')